In [2]:
import sys
# sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")
import numpy as np
import gdsfactory as gf
# TODO
# 1. Change one side of the cell_temp to be comparation with the width
# 2. Different with the width and same width with gap

cell_temp = gf.Component()

2025-03-14 11:06:23.914 | INFO     | kfactory.kcell:show:8777 - klive v0.3.3: Opened file '/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor/build/gds/2201792510.oas'
2025-03-14 11:06:56.349 | INFO     | kfactory.kcell:show:8777 - klive v0.3.3: Opened file '/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor/build/gds/3961826786.oas'


In [ ]:
# pad and lines
pad = gf.Component()

# lines
line_base = gf.Component()
line_half = gf.Component()
# define the cross section  
s1 = gf.Section(width=10, offset=0, layer=(11, 0), name="1", port_names=("o1", "o2"))
X1 = gf.CrossSection(sections=[s1])
s2 = gf.Section(width=0.4, offset=0, layer=(11, 0), name="1", port_names=("o1", "o2"))
X2 = gf.CrossSection(sections=[s2])
# define the path
# define distance
d1 = 90
radius = 20
h1 = 20
d5 = 0.4
p1 = gf.path.straight(length=d1)
p = p1
wg1 = gf.path.extrude(p, X1)
p4 = gf.path.arc(radius=radius, angle=-90)
Xtrans = gf.path.transition(cross_section1=X1, cross_section2=X2, width_type="sine")
curve_transition = gf.path.extrude_transition(p4, Xtrans)

wg1_ref = line_base << wg1
wg2_ref = line_base << curve_transition
wg2_ref.connect("o1", wg1_ref.ports["o2"])

P = gf.Path()
p5 = gf.path.straight(length=h1)
p6 = gf.path.straight(length=d5)
P += p5
P.end_angle += 90  # "Turn" 90 deg (left)
P += p6
wg3 = gf.path.extrude(P, X2)
wg3_ref = line_base << wg3
wg3_ref.connect("o1", wg2_ref.ports["o2"])

# line_base.show()
d = d1+radius+d5
line_base_ref1 = line_half << line_base
line_base_ref2 = line_half << line_base
line_base_ref2.dmirror_x(d)
pad.add_ref(line_half)
pad.add_ref(line_half).drotate(angle=180, center=(2*d, 0))
# pad.show()




In [ ]:
# pads
pad_size = 500
sg_pad = gf.components.pad(size=(pad_size, pad_size), layer=(11, 0))
sg_pad_ref = pad << sg_pad

In [ ]:
# cell
L = 2000
w = [1, 2, 4, 8, 16, 32, 64, 128]

block = gf.Component()
origin = [0, 0]

for i in range(len(w)):
    beam = gf.components.rectangle(size=(w[i], L), layer=(9, 0))
    beam_ref = block << beam
    beam_ref.movex(5000/(len(w)+1)*(i+1)-w[i]/2)
    # length mark
    T = gf.components.text(f"{w[i]}", size=50, layer=(1, 0))
    T_ref = block << T
    T_ref.move(((5000/(len(w)+1)*(i+1)-w[i]/2), -150))
            
# pad
pad = gf.components.rectangle(size=(500, 500), layer=(9, 0))


# block.show()

In [4]:
# backside etching 5mm * 5mm (5743.44um)
backside = gf.Component()
size = 5743.44
diff = (size - 5000) / 2
for i in range(2):
    backside_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backside_ref = backside << backside_temp
    if i == 0:
        backside_ref.move((-diff, -diff))
    elif i == 1:
        backside_ref.move((10000-diff, -diff))

# backside.show()

In [5]:
# structure for each die
# structure for a block
fblock = gf.Component()
for i in range(2):
    block_ref = fblock << block
    if i == 0:
        pass
    elif i == 1:
        block_ref = fblock << block
        block_ref.move((10000, 0))


In [ ]:
# # order
# order = gf.Component()
# text_array = []
# for i in range(2):
#     text_array.append(gf.Component())

# for i in range(2):
#     T = gf.components.text(f"BeL2 {i+1}", size=20, layer=(1, 0))
#     for j in range(4):
#         order_ref = text_array[i] << T
#         if j == 0:
#             order_ref.move((-100, 0))
#         elif j == 1:
#             order_ref.move((-100, 5000))
#         elif j == 2:
#             order_ref.move((5020, 0))
#         else:
#             order_ref.move((5020, 5000))
#     text_array_ref = order << text_array[i]
#     if i == 0:
#         text_array_ref.move((0, 0))
#     elif i == 1:
#         text_array_ref.move((10000, 0))





In [7]:
# backside frame 5mm * 5mm 
backframe = gf.Component()
size = 5000
for i in range(2):
    backframe_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backframe_ref = backframe << backframe_temp
    if i == 0:
        pass
    elif i == 1:
        backframe_ref.move((10000, 0))

# backside.show()

In [ ]:
# boolean operation
# do backside etching - frame
outside = gf.boolean(A = backframe, B = fblock,  operation="not", layer1=(3, 0), layer2=(9, 0), layer=(2, 0))
outside = gf.boolean(A = outside, B = fblock,  operation="not", layer1=(2, 0), layer2=(10, 0), layer=(1, 0))

cell_temp << outside

# add holes to cell_temp
holes = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(8, 0), layer2=(10, 0), layer=(1, 0))
cell_temp << holes
# add holes to cell_temp
pc = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(11, 0), layer2=(30, 0), layer=(2, 0))
cell_temp << pc
cell_temp << backside
# add order
# marker = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(1, 0), layer2=(10, 0), layer=(1, 0))
# cell_temp << marker
# cell_temp << order
# frame
frame1 = gf.components.rectangle(size=(15000, 5000), layer=(20, 0))
cell_temp << frame1
frame2 = gf.components.rectangle(size=(20000, 15830), layer=(21, 0))
frame2_ref = cell_temp << frame2
frame2_ref.move((-2500, -5415))

cell_beam_long2 = gf.Component()
cell_ref = cell_beam_long2 << cell_temp
cell_ref.move((2500, 5415-10000))
cell_beam_long2.show()
# cell_beam_long2.write_gds("mesh.gds")
# cell_beam_long2.plot()